In [11]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")

#Number of Ships skalieren
# Min-Max-Skalierung manuell berechnen
data['Number_of_ships_scaled_current'] = (
    data['Number_of_ships_current'] - data['Number_of_ships_current'].min()
) / (data['Number_of_ships_current'].max() - data['Number_of_ships_current'].min())

#Bewölkung zsmfassen
data['Cloud_ok_current'] = (
    data[['Cloud_Clear_current', 'Cloud_Partly_Cloudy_current']].sum(axis=1) > 0)
# In 0/1 umwandeln
data['Cloud_ok_current'] = data['Cloud_ok_current'].astype(int)

data = data[data["Warengruppe_2"] == 1]
# Liste der gewünschten Spalten
desired_columns = ['Datum',
            'Umsatz',
            #'Temp_Very_Cold', 
            'Temp_Cold_current', 
            'Temp_Mild_current', 
            'Temp_Warm_current', 
            'Temp_Hot_current', 
            #'Cloud_Clear', 
            #'Cloud_Partly_Cloudy', 
            'Cloud_ok_current',
            'Cloud_Cloudy_current', 
            #'Wind_Light', 
            #'Wind_Moderate', 
            #'Wind_ok',
            #'Wind_Strong', 
            #'Weather_Good', 
            #'Weather_Light_Issues', 
            #'Weather_Moderate', 
            #'Weather_Severe', 
            'KielerWoche_current', 
            #'Montag_current', 
            #'Dienstag_current', 
            #'Mittwoch_current', 
            #'Donnerstag_current', 
            #'Freitag_current',
            'Samstag_current',
            'Sonntag_current',
            'VPI_current',
            #'Number_of_ships',
            'Number_of_ships_scaled_current',
            #'Ship',
            #'Heimspiel', --> weglassen
            'Feiertag_current',
            'is_holiday_current',
            'Weihnachtsmarkt_current',
            #'Markt',
            'Ostertag_current',
            'Silvester_current',
            #'Werktag_current',
            #'Frühling',
            'Sommer_current',
            #'Herbst',
            'Winter_current',
            'wetter_sehr_schlecht_current',
            'wetter_sehr_schön_current',
            #'Monat_1',
            #'Monat_2',
            'Monat_7_current',
            'Monat_1_current',
            #'Monat_11',
            #'zwischen_den_jahren',
            #'wetter_sehr_schön_prev',
            #'Temp_Hot_prev' --> lag hat sich hier nicht ausgezahlt
            'is_holiday_next'
            ]

# Wähle nur die gewünschten Spalten aus
data = data[desired_columns]

data.head()  # Print first few rows to verify

,Datum,Umsatz,Temp_Cold_current,Temp_Mild_current,Temp_Warm_current,Temp_Hot_current,Cloud_ok_current,Cloud_Cloudy_current,KielerWoche_current,Samstag_current,...,Weihnachtsmarkt_current,Ostertag_current,Silvester_current,Sommer_current,Winter_current,wetter_sehr_schlecht_current,wetter_sehr_schön_current,Monat_7_current,Monat_1_current,is_holiday_next
1819,2013-07-01,535.856285,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1.0
1820,2013-07-02,546.780787,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,1.0
1821,2013-07-03,427.343259,0,0,1,0,0,1,0,0,...,0,0,0,1,0,1,0,1,0,1.0
1822,2013-07-04,454.859641,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1.0
1823,2013-07-05,492.818804,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,1.0


In [12]:
# Handle missing values by removing rows with any missing values
data = data.dropna()

# Data split Zeitreihen
# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'

# Split the data based on the date thresholds
train_data = data[data['Datum'] <= train_end_date]
vali_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[data['Datum'] > validation_end_date]

# Check the dimensions of the datasets
print("Training dataset dimensions:", train_data.shape)
print("Validation dataset dimensions:", vali_data.shape)
print("Test dataset dimensions:", test_data.shape)

# Separating features and labels
training_features = train_data.drop('Umsatz', axis=1).drop('Datum', axis=1)
validation_features = vali_data.drop('Umsatz', axis=1).drop('Datum', axis=1)
test_features = test_data.drop('Umsatz', axis=1).drop('Datum', axis=1)

training_labels = train_data[['Umsatz']]
validation_labels = vali_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print()
print()
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)

Training dataset dimensions: (1462, 25)
Validation dataset dimensions: (357, 25)
Test dataset dimensions: (0, 25)


Training features dimensions: (1462, 23)
Validation features dimensions: (357, 23)
Test features dimensions: (0, 23)

Training labels dimensions: (1462, 1)
Validation labels dimensions: (357, 1)
Test labels dimensions: (0, 1)


In [13]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data_Broetchen"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")